In [1]:
!pip install -qq transformers genomic-benchmarks datasets

     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 362 kB 73.7 MB/s 
     |████████████████████████████████| 6.6 MB 67.0 MB/s 
     |████████████████████████████████| 101 kB 14.8 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 2.3 MB 78.4 MB/s 
     |████████████████████████████████| 271 kB 87.2 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
     |████████████████████████████████| 140 kB 2.4 MB/s 
     |████████████████████████████████| 212 kB 69.7 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
     |████████████████████████████████| 144 kB 68.1 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
### Parameters
MODEL_NAME = "Vlasta/DNADebertaK7" #Original DNABert model
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 7
STRIDE = 1

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 1

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_nontata_promoters', 0)]

BATCH_SIZE = 8
ACCUMULATION = 8

LEARNING_RATE = 1e-5
EPOCHS = 4
RUNS = 5

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/RandomizedDNADebertaK7.csv'

## Download benchmark datasets and tokenizer

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [3]:
from itertools import product

alphabet = ('A', 'C', 'T', 'G')
vocab = list(map(''.join, product(alphabet, repeat=K)))

tokenizer.add_tokens(vocab)

16384

In [4]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version, use_cloud_cache=True)

benchmark_root = Path(BENCHMARKS_FOLDER)

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 7109, 16136, 19473, 16440, 4308, 4930, 7417, 17366, 8010, 19737, 17495, 8527, 5422, 9387, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [6]:
import pandas as pd
import numpy as np
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
        model_cls.deberta.encoder.layer.apply(model_cls.deberta._init_weights)

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2, gradient_accumulation_steps=ACCUMULATION,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, seed=randrange(1,10001), report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))
        outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
        outputs_df.to_csv(OUTPUT_PATH, index=False)




  0%|          | 0/4 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f47a84e3710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/100000 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

...

***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/Vlasta/DNADebertaK7/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3fa25099222ee8392148633f9da2ccd667d7c3946c0cb204e54570049d533aca.c59d09939f4655d9540dd690f39534d728c6afa978a255af7840e17b5db6c906
Model config DebertaConfig {
  "_name_or_path": "Vlasta/DNADebertaK7",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 51

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.385674,0.840049,0.847911
1,0.472300,0.292250,0.886761,0.892350
2,0.301900,0.254090,0.905690,0.910990
3,0.206600,0.253161,0.908568,0.914298


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/Vlasta/DNADebertaK7/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3fa25099222ee8392148633f9da2ccd667d7c3946c0cb204e54570049d533aca.c59d09939f4655d9540dd690f39534d728c6afa978a255af7840e17b5db6c906
Model config DebertaConfig {
  "_name_or_path": "Vlasta/DNADebertaK7",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 51

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.385638,0.832743,0.829053
1,0.465900,0.281141,0.887868,0.892131
2,0.287700,0.246649,0.906686,0.911958
3,0.200900,0.249288,0.908457,0.914733


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [7]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.908760,0.908109,1593.7100
1,demo_coding_vs_intergenomic_seqs,0.910360,0.910090,1597.5654
2,demo_coding_vs_intergenomic_seqs,0.909280,0.908282,1597.7165
3,demo_coding_vs_intergenomic_seqs,0.908760,0.908117,1596.3733
4,demo_coding_vs_intergenomic_seqs,0.909840,0.909054,1597.9557
5,demo_human_or_worm,0.961680,0.961766,1602.1379
6,demo_human_or_worm,0.960400,0.960605,1599.1343
7,demo_human_or_worm,0.960520,0.960600,1605.7688
8,demo_human_or_worm,0.960680,0.960549,1608.9437
9,demo_human_or_worm,0.960680,0.960779,1608.4374


In [8]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy                  f1            \
                                      mean       sem      mean       sem   
dataset                                                                    
demo_coding_vs_intergenomic_seqs  0.909400  0.000312  0.908731  0.000382   
demo_human_or_worm                0.960792  0.000228  0.960860  0.000230   
human_enhancers_cohn              0.736903  0.001149  0.740870  0.001791   
human_nontata_promoters           0.908944  0.000506  0.914633  0.000336   

                                 train_runtime            
                                          mean       sem  
dataset                                                   
demo_coding_vs_intergenomic_seqs    1596.66418  0.787381  
demo_human_or_worm                  1604.88442  1.877376  
human_enhancers_cohn                 577.15112  0.044168  
human_nontata_promoters              596.25504  0.862439

In [9]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)

In [10]:
model_cls.deberta.encoder.layer.apply(model_cls.deberta._init_weights)

ModuleList(
  (0): DebertaLayer(
    (attention): DebertaAttention(
      (self): DisentangledSelfAttention(
        (in_proj): Linear(in_features=768, out_features=2304, bias=False)
        (dropout): StableDropout()
      )
      (output): DebertaSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): DebertaLayerNorm()
        (dropout): StableDropout()
      )
    )
    (intermediate): DebertaIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): DebertaOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
  )
  (1): DebertaLayer(
    (attention): DebertaAttention(
      (self): DisentangledSelfAttention(
        (in_proj): Linear(in_features=768, out_features=2304, bias=False)
        (dropout): StableDropout()
      )
      (output): DebertaSe

In [11]:
model_cls.deberta.encoder.layer[1].intermediate.dense.weight

Parameter containing:
tensor([[-0.0108, -0.0033,  0.0046,  ...,  0.0033,  0.0180,  0.0032],
        [-0.0306, -0.0175,  0.0011,  ..., -0.0134,  0.0182, -0.0247],
        [-0.0026, -0.0368, -0.0020,  ...,  0.0002, -0.0233,  0.0100],
        ...,
        [-0.0158, -0.0048,  0.0221,  ...,  0.0244,  0.0183,  0.0162],
        [ 0.0130,  0.0132, -0.0029,  ..., -0.0278,  0.0005, -0.0168],
        [ 0.0354,  0.0184, -0.0151,  ...,  0.0071,  0.0070,  0.0044]],
       device='cuda:0', requires_grad=True)